In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from chapter07.td import td_on_policy_backward, sarsa_eligibility_traces
from chapter07.walk import WalkRightEnv
from plotly.express import pd
import gymnasium as gym
from tqdm import tqdm
import numpy as np

In [ ]:
from plotly.graph_objs import Figure

In [93]:
size = 15

In [94]:
env = gym.make('WalkRightEnv-v0', size=size)


In [95]:
agent = sarsa_eligibility_traces(env, 100, 1000, 0.1, lambda_val=0.9, gamma=0.8, alpha=0.9,
                                 replacing_trace=True, verbose=False)
agent.Q

C:\Users\sam24\.conda\envs\machine_learning\lib\site-packages\gymnasium\utils\passive_env_checker.py:218: UserWarning:

WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'int'>`

C:\Users\sam24\.conda\envs\machine_learning\lib\site-packages\gymnasium\utils\passive_env_checker.py:242: DeprecationWarning:

WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 



array([[0.0429383 , 0.05249366],
       [0.04798647, 0.06579544],
       [0.0539115 , 0.08250554],
       [0.07890179, 0.10350515],
       [0.09198012, 0.12990261],
       [0.09699023, 0.16308983],
       [0.16432962, 0.204811  ],
       [0.20870449, 0.25724514],
       [0.25150957, 0.32310328],
       [0.3257909 , 0.40573996],
       [0.2962959 , 0.50927574],
       [0.50776448, 0.63872325],
       [0.63864286, 0.80011682],
       [0.8045818 , 1.00008676],
       [0.        , 0.        ]])

In [101]:
alphas = np.arange(0.1, 1.1, 0.1)
lambdas = np.arange(0, 1.1, 0.1)
rmse = np.zeros((len(alphas), len(lambdas)))
for i, alpha in tqdm(enumerate(alphas), total=len(alphas)):
    for j, lambda_val in enumerate(lambdas):
        v_hat = []
        for _ in range(20):
            agent = sarsa_eligibility_traces(env, 10, 1000, 0.1, lambda_val=lambda_val, alpha=alpha, replacing_trace=False, gamma=0.8, verbose=False)
            v_hat.append(np.mean(np.diff(agent.Q, axis=1)))
        v_hat = np.mean(v_hat, axis=0)
        rmse[i, j] = v_hat

rsme_df = pd.DataFrame(rmse, index=alphas, columns=lambdas)
rsme_df.index.names = ['alpha']

fig = Figure()
for c, col in rsme_df.T.items():
    fig.add_scatter(x=col.index, y=col, name=round(c, 2))

fig.update_layout(yaxis_title='score', xaxis_title='lambda', legend_title='alpha')
fig.show()

100%|██████████| 10/10 [00:44<00:00,  4.46s/it]


In [100]:
replacing_traces = [True, False]
lambdas = np.arange(0, 1.1, 0.1)
rmse = np.zeros((len(replacing_traces), len(lambdas)))
for i, replacing_trace in tqdm(enumerate(replacing_traces), total=len(replacing_traces)):
    for j, lambda_val in enumerate(lambdas):
        v_hat = []
        for _ in range(20):
            agent = sarsa_eligibility_traces(env, 10, 1000, 0.1, gamma=0.8,
                                             lambda_val=lambda_val, alpha=rsme_df.mean(axis=1).nlargest(1).index[0],
                                             replacing_trace=replacing_trace, verbose=False)
            v_hat.append(np.mean(np.diff(agent.Q, axis=1)))
        v_hat = np.mean(v_hat, axis=0)
        rmse[i, j] = v_hat
rsme_df = pd.DataFrame(rmse, index=replacing_traces, columns=lambdas)
rsme_df.index.names = ['replacing_trace']
rsme_df = rsme_df.T

fig = Figure()
for c, col in rsme_df.items():
    fig.add_scatter(x=col.index, y=col, name=round(c, 2))

fig.update_layout(yaxis_title='score', xaxis_title='lambda', legend_title='replacing trace')
fig.show()

100%|██████████| 2/2 [00:08<00:00,  4.38s/it]
